<a href="https://colab.research.google.com/github/willianjferreira/trabalho_edpa_2019/blob/master/GP_Willian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importação de bibliotecas

In [0]:
import random
import copy

Definição das Primitivas

In [0]:
terminais = ['0','1','2','3','4','5','6','7','8','9','X','Y']
operadores = ['+','-','*','/']

Inicialização de variáveis, listas e constantes

In [0]:
num_individuos= 1200
num_entradas= 5000
num_geracoes = 15
taxa_de_cruzamento = 0.7    #  70% = 0.7
taxa_de_mutacao = 0.03      #   3% = 0.03
t = int(num_individuos/2)
funcao_referencia = '(X+4) * (3-Y)'   #Função Objetivo
entradas = []
saidas = []

Geração dos dados de entrada

In [0]:
for e in range(num_entradas):
  entradas.append(random.randint(-300,300))
print('entradas:', entradas)

entradas: [-2, 296, 184, 282, 3, 144, -250, -97, -60, -18, -161, 249, 106, 150, 291, -182, 79, 103, 208, -108, -191, -10, 267, -49, -184, 138, 77, 271, 65, 17, 124, -235, 197, 43, -259, -259, -286, -226, 252, 282, 132, 38, 153, -74, 213, -246, 1, 254, -263, 262, 138, 266, -25, 255, -230, 52, 296, -82, -206, 221, 30, 15, 176, 35, -222, -263, 279, 195, -25, 77, 162, -160, -67, 227, 45, -44, -199, -25, 46, -284, 149, 222, -230, 144, 130, -15, -191, -240, 289, -234, 249, 37, -45, 46, -49, -257, -4, 24, 139, -131, -12, 163, -143, 223, -162, 163, 161, 14, 270, -32, 138, 205, -211, -240, -231, 18, -132, 61, 175, -151, 164, -18, 180, -139, 157, 151, -252, 203, 181, 65, 41, 111, 286, -256, -69, 270, -34, 41, -10, -85, -48, -65, -59, -200, 52, 261, 39, 99, 279, -168, 22, 224, 136, -297, 4, -108, -79, -248, -292, -92, -161, -299, -88, 217, -16, 183, -294, -225, -125, -121, 98, 292, 250, 135, 145, -10, -143, 175, 66, 11, -5, -182, -151, 111, 143, 179, 249, 255, -49, 170, 125, -237, -107, -258, -17

Geração dos dados de Saida a partir da entrada e da função referência

In [0]:
def executa_equacao(equacao):
  saidas=[]
  for i in range(len(entradas)):
    X= entradas[i]
    Y= entradas[-(i+1)]
    # print('---- i=',i)
    # print('X=',X,'','Y=',Y)
    try:
      saidas.append(eval(equacao))
    except ZeroDivisionError:
      saidas.append(1000)
    return saidas
  #   print(funcao_referencia)
  #   print(saidas[i])
  # print(saidas)

In [0]:
saidas = executa_equacao(funcao_referencia)

Métodos de criaçao dos indivíduos (programas - representados em forma de árvores)

In [0]:
class Tree :
  def __init__(self, primitiva, left=None, right=None) :
    self.primitiva = primitiva
    self.left  = left
    self.right = right

  def __str__(self) :
    return str(self.primitiva)

class individuo:
  
    def __init__(self):
        self.raiz = None
        self.fenotipo = 'fenot'
        self.fitness = 0.0
    
    def __str__(self) :
      return (str(self.fenotipo) +' '+ str(self.fitness))

    def printTreeInorder(self) :
        printTree(self.raiz)

    def calcula_fitness(self):
        saida_list = executa_equacao(self.fenotipo)
        erro = sum(abs(a - b) for a, b in zip(saida_list, saidas))
        self.fitness = erro
        # print(self.fitness)
    
    def Tree4String(self) :
        # self.fenotipo= t4s(self.raiz, '')
        self.fenotipo= printTree_deuslirio(self.raiz)
        # print(self.fenotipo)

def printTree(tree) :
    if tree == None : return
    printTree(tree.left)
    print (tree.primitiva, end="")
    printTree(tree.right)

def printTree_deuslirio(tree):
  if (tree.left==None and tree.right == None): 
    return tree.primitiva[0]
  else:
    return "("+ printTree_deuslirio(tree.left)+")" + tree.primitiva[0] + "(" + printTree_deuslirio(tree.right) + ")"

**Função para determinar o melhor indivídio em uma População**

In [0]:
def torneio_best_ind(individuos_list):
  
  best_indiv = individuos_list[0]

  for indiv in (individuos_list):
    if indiv.fitness < best_indiv.fitness:
      best_indiv = indiv
  return best_indiv
    

**Função para selecionar indivíduos e compor a próxima POPULAÇÃO - Método de seleção: TORNEIO**

In [0]:
def torneio_selecao(individuos_list):
  
  T=t #tamanho do torneio

  individuos_selecionados = []
  for k in range(T):
    #seleciona aleatoriamente t individuos da população inicial
    individuos_torneio = random.sample(individuos_list,T) 

    #escolhe dentre os N individuos o melhor deles
    best_indiv = torneio_best_ind(individuos_torneio)
    individuos_selecionados.append(best_indiv)
    individuos_list.remove(best_indiv)
    T=T-1
    # repita o processo até o valor de T (TAMANHO DO TORNEIO)

  return individuos_selecionados

**Funçao de MUTAÇÃO DE INDIVÍDUOS.**

um programa é selecionado conforme a taxa de mutação definida TXM e um de seus nós é escolhido aleatoriamente. A sub-árvore cuja raiz é o nó selecionado é então eliminada e substituída por uma nova árvore gerada aleatoriamente.

In [0]:
def mutacao(individuos_list):

  max = len(individuos_list)
  arvore_mutacao=[]

  k=0
  n_mut = 0

  while k < max:
  
    txm = random.randint(0, 100)
  
    if((txm/100) <= taxa_de_mutacao):  # Se valor gerado aleatoriamente é <= tx de mutação, então realiza a mutação
      
      ind = individuo()
      ind.raiz = criar_arvore()
      
      arvore_mutacao.append(ind)

      arvore_mutacao[n_mut].Tree4String()
 
      if (txm % 2 == 0): # Se txm é PAR, realiza mutação na subarvore direita
        individuos_list[k].fenotipo = copy.deepcopy(individuos_list[(k)].fenotipo[0:9]+
                                                  individuos_list[k].fenotipo[9:10]+
                                                  arvore_mutacao[n_mut].fenotipo[0:9])
        
      else: # Se txm é IMPAR, realiza mutação na subarvore esquerda
        individuos_list[k].fenotipo = copy.deepcopy(arvore_mutacao[n_mut].fenotipo[0:9]+
                                          individuos_list[k].fenotipo[9:10]+
                                          individuos_list[k].fenotipo[10:19])
      n_mut +=1
      
      individuos_list[k].calcula_fitness()

      
    else: #Não houve mutacao

      individuos_list[k].fenotipo = copy.deepcopy(individuos_list[k].fenotipo[0:9]+
                                          individuos_list[k].fenotipo[9:10]+
                                          individuos_list[k].fenotipo[10:19])
      individuos_list[k].calcula_fitness()

    k+=1
  
  return individuos_list

**Funçao de CRUZAMENTO DE INDIVÍDUOS.**

Dois indivíduos (programas) são selecionados e são recombinados para gerar outros dois programas. Um ponto aleatório de cruzamento é escolhido em cada programa-pai e as árvores abaixo destes pontos são trocadas. 

In [0]:
def cruza_populacao(individuos_list):

  max = len(individuos_list)

  k=1
  while k < max:

    txc = random.randint(0, 100)
  
    if(txc/100 <= taxa_de_cruzamento): # Se valor gerado aleatoriamente é <= tx de cruzamento, então realiza a crossover
      
      subarvore_esq_de_k_previous= copy.deepcopy(individuos_list[k-1].fenotipo[0:9])
      subarvore_esq_de_k = copy.deepcopy(individuos_list[(k)].fenotipo[0:9])

      individuos_list[k-1].fenotipo = copy.deepcopy(subarvore_esq_de_k+
                                                    individuos_list[k-1].fenotipo[9:10]+
                                                    individuos_list[k-1].fenotipo[10:19])
      
      
      individuos_list[k-1].calcula_fitness()
      
      individuos_list[k].fenotipo = copy.deepcopy(subarvore_esq_de_k_previous+
                                                    individuos_list[k].fenotipo[9:10]+
                                                    individuos_list[k].fenotipo[10:19])
      
      individuos_list[k].calcula_fitness()
    
    else: # NÃO HOUVE crossover
    
      individuos_list[k-1].fenotipo = copy.deepcopy(individuos_list[k-1].fenotipo)
      
      individuos_list[k-1].calcula_fitness()

      individuos_list[k].fenotipo = copy.deepcopy(individuos_list[k].fenotipo )

      individuos_list[k].calcula_fitness()     

    k+=2

  return individuos_list


**GERACÃO DE INDIVÍDUOS (PROGRAMAS) - POPULAÇÃO INICIAL**

Geração de Indivíduos utilizando o Método Full (todos nós folhas estão no mesmo
nível).

In [0]:
populacao_inicial=[]

def criar_arvore():

  arv = Tree( random.sample(operadores,k=1),  #raiz
            Tree( random.sample(operadores,k=1), Tree(random.sample(terminais,k=1)), Tree(random.sample(terminais,k=1))), #sub-arvore esquerda
            Tree( random.sample(operadores,k=1), Tree(random.sample(terminais,k=1)), Tree(random.sample(terminais,k=1))) ) #sub-arvore direita
  return arv

for n in range(num_individuos):
  ind = individuo()
  ind.raiz = criar_arvore()
  # print(ind.fenotipo)
  populacao_inicial.append(ind)
  # print('\n Individuo', n)
  # populacao_inicial[n].printTreeInorder()
  populacao_inicial[n].Tree4String()
  populacao_inicial[n].calcula_fitness()

**Programa Principal**

![alt text](http://www.soumaisintegra.com.br/img/gpfluxo.png)

In [0]:
selecionados=[]
cruzados = []
mutados = []
melhores_individuos=[]

##------ INICIO - CARREGA POPULAÇÃO INICIAL ------------------
selecionados= copy.deepcopy(populacao_inicial)  

fitness = -1
g=0
 ##------ EXECUTA ENQUANTO NÃO ATINGE OBJETIVO (Num gerações ou Fitness = 0) ---------
while ( g < (num_geracoes) and (fitness != 0) ):
  
  print('--GERAÇAO ',g,'--')
  # print('Função Referencia >>',funcao_referencia)

  ##------ CALCULO DA APTIDÃO ------------------
  melhores_individuos.append(torneio_best_ind(selecionados))
  print('Melhor Individuo >> ',melhores_individuos[g])
  fitness = copy.deepcopy(melhores_individuos[g].fitness)

  ##------ SELEÇÃO OPERADORES GENETICOS------------------
  selecionados = copy.deepcopy(torneio_selecao(selecionados))
  cruzados = copy.deepcopy(cruza_populacao(selecionados))
  mutados = copy.deepcopy(mutacao(cruzados))

  ##--- ATUALIZA POPULAÇÃO PARA CALCULO DE APTIDÃO--------
  selecionados=copy.deepcopy(mutados)
  g +=1

print('\n ************* CRITÉRIO DE PARADA ATINGIDO! *************')

# for x in melhores_individuos:
#   print(x)

print('Função Referencia  ou Função Objetivo >>>>>>>>',funcao_referencia)
print('O MELHOR Individuo de todas as geracoes foi >> ',torneio_best_ind(melhores_individuos))
print('e foi encontrado na GERACAO >> ',melhores_individuos.index(torneio_best_ind(melhores_individuos)))

--GERAÇAO  0 --
Função Referencia >> (X+4) * (3-Y)
Melhor Individuo >>  ((1)-(Y))+((4)-(Y)) 1
--GERAÇAO  1 --
Função Referencia >> (X+4) * (3-Y)
Melhor Individuo >>  ((2)+(0))+((Y)*(X)) 4
--GERAÇAO  2 --
Função Referencia >> (X+4) * (3-Y)
Melhor Individuo >>  ((9)-(Y))-((2)+(Y)) 1
--GERAÇAO  3 --
Função Referencia >> (X+4) * (3-Y)
Melhor Individuo >>  ((X)*(8))*((4)+(8)) 0

 ************* CRITÉRIO DE PARADA ATINGIDO! *************
O MELHOR Individuo de todas as geracoes foi >>  ((X)*(8))*((4)+(8)) 0
e foi encontrado na GERACAO >>  3
